In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# install dependencies: 
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

     |████████████████████████████████| 276kB 24.2MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44074 sha256=174a97404cfde9c39d671ef6d12ded461566e95461659950d2586fceaf50b2fb
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
1.8.1+cu101 True
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [3]:
# install detectron2: (Colab has CUDA 10.1 + torch 1.8)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
import torch
assert torch.__version__.startswith("1.8")   # need to manually install torch 1.8 if Colab changes its default version
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
     |████████████████████████████████| 6.2MB 622kB/s 
     |████████████████████████████████| 51kB 8.4MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.3.post20210317-cp37-none-any.whl size=58543 sha256=f6d39e67d7b0385e54545f9a2a4bf1acaa7d2923f6e1a0afb2a2dd68a4c1f865
  Stored in directory: /root/.cache/pip/wheels/d2/ee/3a/5c531df777c03d8c67f22c65f97d6f75321087482d05a9b218
Successfully built fvcore


In [4]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.engine import DefaultTrainer
import os
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import (
    CityscapesInstanceEvaluator,
    CityscapesSemSegEvaluator,
    COCOEvaluator,
    COCOPanopticEvaluator,
    DatasetEvaluators,
    LVISEvaluator,
    PascalVOCDetectionEvaluator,
    SemSegEvaluator,
    verify_results,
    inference_on_dataset,
)
from detectron2.data import (
    MetadataCatalog,
    build_detection_test_loader,
    build_detection_train_loader,)

In [5]:
!ls

drive  sample_data


In [6]:
register_coco_instances('dataset', {}, \
                        '/content/drive/MyDrive/evaluator/trainval.json', \
                        '/content/drive/MyDrive/evaluator/')

In [7]:
cfg = get_cfg()
cfg.merge_from_file('/content/drive/MyDrive/model_files/doclynet.yaml')
# cfg.merge_from_file('/content/doclaynet2.yaml')
cfg.DATASETS.TRAIN = ('dataset',)
cfg.DATASETS.TEST = ('dataset',)
cfg.MODEL.WEIGHTS = '/content/drive/MyDrive/model_files/doclaynet_v2_1.pth'
# cfg.MODEL.WEIGHTS = '/content/doclaynet_v2_1.pth'
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST=0.7
os.makedirs('./output', exist_ok=True)

In [12]:
class Trainer(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg)

trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)

# evaluator = COCOEvaluator("dataset", cfg, tasks=("bbox"), False, output_dir="./output")
evaluator = COCOEvaluator("dataset", cfg, False, output_dir="./output")
val_loader = build_detection_test_loader(cfg, "dataset")
inference_on_dataset(trainer.model, val_loader, evaluator)

[05/18 18:57:41 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


WARNING [05/18 18:58:09 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
WARNING [05/18 18:58:09 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[05/18 18:58:09 d2.data.datasets.coco]: Loaded 47 images in COCO format from /content/drive/MyDrive/evaluator/trainval.json
WARNING [05/18 18:58:09 d2.data.datasets.coco]: Filtered out 475 instances without valid segmentation. There might be issues in your dataset generation process. A valid polygon should be a list[float] with even length >= 6.
[05/18 18:58:09 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[05/18 18:58:09 d2.data.common]: Serializing 47 elements to byte tensors and concatenating them all ...
[05/18 18:58:09 d2.data.common]: Serialized dataset takes 0.0

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[05/18 18:58:20 d2.evaluation.evaluator]: Inference done 11/47. 0.2554 s / img. ETA=0:00:24
[05/18 18:58:26 d2.evaluation.evaluator]: Inference done 19/47. 0.2563 s / img. ETA=0:00:19
[05/18 18:58:32 d2.evaluation.evaluator]: Inference done 27/47. 0.2575 s / img. ETA=0:00:13
[05/18 18:58:37 d2.evaluation.evaluator]: Inference done 34/47. 0.2593 s / img. ETA=0:00:09
[05/18 18:58:43 d2.evaluation.evaluator]: Inference done 42/47. 0.2583 s / img. ETA=0:00:03
[05/18 18:58:46 d2.evaluation.evaluator]: Total inference time: 0:00:29.845174 (0.710599 s / img per device, on 1 devices)
[05/18 18:58:46 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:10 (0.258376 s / img per device, on 1 devices)
[05/18 18:58:46 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[05/18 18:58:46 d2.evaluation.coco_evaluation]: Saving results to ./output/coco_instances_results.json
[05/18 18:58:46 d2.evaluation.coco_evaluation]: Evaluating predictions with unofficial COCO API...


TypeError: ignored

In [ ]:
!cat /usr/local/lib/python3.7/dist-packages/pycocotools/coco.py